### Silver Layer Pipeline

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

#### Define Bronze-to-Silver Table _stage_bookings_

- This Delta Live Table (DLT) loads streaming data from the bookings folder in the Bronze layer
- It uses spark.readStream to continuously read Delta files and registers the output as a managed DLT table

In [0]:
@dlt.table(
    name = "stage_bookings"
)
def stage_bookings():
    df = spark.readStream.format("delta")\
        .load("/Volumes/flights/bronze/bronze_volume/bookings/data/")
    return df

### Apply Transformations to Bookings

- This DLT view applies transformations to the _stage_bookings_ data, including type casting, adding timestamps, and dropping _rescued_data columns

In [0]:
@dlt.view(
    name = "transformations_bookings"
)
def transformations_bookings():
    df = spark.readStream.table("stage_bookings")
    df = df.withColumn("amount", col("amount").cast(DoubleType()))\
        .withColumn("modified_date", current_timestamp())\
        .withColumn("booking_date", to_date(col("booking_date")))\
        .drop("_rescued_data")
    return df

### Define Data Quality Rules

- A dictionary _rules_ is defined, which includes constraints for data quality checks (e.g., booking_id and passenger_id must not be NULL)

In [0]:
rules = {
    "rule1": "booking_id IS NOT NULL",
    "rule2": "passenger_id IS NOT NULL"
}

### Create Silver Table with Data Expectations

- This DLT table _silver_bookings_ reads from the transformed view and enforces all data quality rules via @dlt.expect_all_or_drop
- Rows that violate rules are dropped

In [0]:
@dlt.table(
    name = "silver_bookings"
)

@dlt.expect_all_or_drop(rules)

def silver_bookings():
    df = spark.readStream.table("transformations_bookings")
    return df

### Query Silver Layer

In [0]:
%sql
SELECT *
FROM flights.silver.silver_airports
LIMIT 10;

airport_id,airport_name,city,country,modified_date
A012,East Nancy International Airport,Crosston,Romania,2025-07-10T19:49:14.572Z
A044,Cordovaview International Airport,Shannonport,Gabon,2025-07-10T19:49:14.572Z
A047,New Michaelland International Airport,Sanchezberg,Hong Kong,2025-07-10T19:49:14.572Z
A050,Paultown International Airport,North Richardmouth,France,2025-07-10T19:49:14.572Z
A009,New Joshuafurt International Airport,Port Jamiehaven,Pitcairn Islands,2025-07-10T19:49:14.572Z
A023,Maryfurt International Airport,Lake Kennethfurt,Panama,2025-07-10T19:49:14.572Z
A043,Kaylaland International Airport,Schultzhaven,Sweden,2025-07-10T19:49:14.572Z
A022,Blankenshipport International Airport,Sherryville,Bhutan,2025-07-10T19:49:14.572Z
A032,East Robinmouth International Airport,Sharonstad,Poland,2025-07-10T19:49:14.572Z
A005,East Aaron International Airport,Davishaven,Monaco,2025-07-10T19:49:14.572Z
